In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.data import Dataset

In [2]:
df_sub = pd.read_csv(Path.cwd() / 'data' / 'sample_submission.csv')
df_train = pd.read_csv(Path.cwd() / 'data' / 'train.csv')
df_test = pd.read_csv(Path.cwd() / 'data' / 'test.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_train.drop_duplicates(inplace=True)

In [4]:
df_train.userid.max()

127495

In [57]:
df_test.userid.max()

127495

In [5]:
train_data, test_data = train_test_split(df_train, random_state=32, shuffle=True)

In [9]:
ratings_coo = sparse.coo_matrix((df_train['rating'].astype(int),
                                 (df_train['userid'],
                                  df_train['itemid'])))

In [10]:
ratings_coo.shape

(127496, 41320)

In [18]:
len(df_train['overall'].astype(int))

828935

In [21]:
len(df_train)

828935

In [11]:
user_feature_coo = sparse.coo_matrix((df_train['overall'].astype(int),
                                     (df_train['userid'],
                                     [0]*len(df_train))))

In [12]:
user_feature_coo.shape

(127496, 1)

In [39]:
ratings_coo.shape

(127496, 41320)

In [13]:
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 30 #число параметров вектора 
NUM_EPOCHS = 25 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo,
                  user_features=user_feature_coo,
                  epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [14]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values,
                      user_features=user_feature_coo
                     )

In [15]:
roc_auc_score(test_data.rating, preds)

0.761374981408911

In [22]:
preds_sub = model.predict(df_test.userid.values,
                      df_test.itemid.values, user_features=user_feature_coo)

In [23]:
preds_sub.min(), preds_sub.max()

(-51.21615982055664, 409.8995666503906)

In [18]:
normalized_preds = (preds_sub - preds_sub.min())/(preds_sub - preds_sub.min()).max()

In [19]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [20]:
df_sub['rating']= normalized_preds

In [21]:
df_sub.to_csv('submission_base_log.csv', index=False)